### Import

In [1]:
import os
import itertools
import time
import pandas as pd
import numpy as np

from dfg_rating.model import factory
import dfg_rating.viz.jupyter_widgets as DFGViz

from dfg_rating.model.rating.elo_rating import ELORating
from dfg_rating.model.network.base_network import WhiteNetwork
from dfg_rating.model.forecast.true_forecast import LogFunctionForecast
from dfg_rating.model.rating.controlled_trend_rating import ControlledTrendRating, ControlledRandomFunction


from dfg_rating.model.betting.betting import FixedBetting
from dfg_rating.model.bookmaker.base_bookmaker import BaseBookmaker
from dfg_rating.model.evaluators.accuracy import RankProbabilityScore, Likelihood, ProbabilityDifference, ProbabilityPointer, FavouriteProbability
from dfg_rating.model.evaluators.profitability import BettingReturnsEvaluator
from dfg_rating.model.evaluators.base_evaluators import BettingActivity

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.width = None
pd.set_option('display.float_format', lambda x: '%.5f' % x)

### Loading real data

In [2]:
data_football_national = pd.read_csv(os.path.join('..', '..', '..', 'data', 'real', 'Data_Football_National.csv'),sep = ";")
#data_football_national = pd.read_csv("C:/Users/Dell-PC/Documents/Projekte/dfg-rating/data/real/Data_Football_National.csv")

New mapping available:
 - ts_format: Timestamp format
 - winner with result and translation to (home, draw, away).

In [3]:
football_network = WhiteNetwork(
    data=data_football_national,
    #node1 = away
    mapping={
        "node1": {
            "id": "AwayID",
            "name": "AwayTeam",
        },
        "node2": {
            "id": "HomeID",
            "name": "HomeTeam",
        },
        "day": "Date",
        "dayIsTimestamp": True,
        "ts_format": "%d.%m.%Y",
        "season": "Season",
        "winner": {
            "result": "ResultFT",
            "translation": {
                "H": "home",
                "D": "draw",
                "A": "away"
            }
        },
        "round": "day",
        "odds": {
            "maximumodds": {
                "home": "OddsHomeMax",
                "draw": "OddsDrawMax",
                "away": "OddsAwayMax"
            },
            "averageodds": {
                "home": "OddsHomeAvg",
                "draw": "OddsDrawAvg",
                "away": "OddsAwayAvg"
            },
        },
        "bets": {}
    }
)

Network loaded correctly


### Adding ratings to the loaded network

In [12]:
football_network.add_rating(
    rating=ELORating(
        trained=True, 
        rating_name='player_rating',
        **{
            'param_c' : 10,
            'param_d' : 400,
            'param_k' : 50,
            'param_w' : 50
        }
    ),
    rating_name='player_rating'
)

Rating explorer (Only useful the first chart, ignore the others)

In [13]:
app = DFGViz.RatingsExplorer(
    network=football_network
)

In [14]:
app.run('inline')

### Adding a forecast from the added rating

In [7]:
football_network.add_forecast(
    forecast=LogFunctionForecast(outcomes=['home', 'draw', 'away'], coefficients=[-1.2, 0.0], beta_parameter=0.006),
    forecast_name='player_forecast',
    base_ranking='player_rating'
)
football_network.add_forecast(
    forecast=LogFunctionForecast(outcomes=['home', 'draw', 'away'], coefficients=[-0.9, 0.3], beta_parameter=0.006),
    forecast_name='true_forecast',
    base_ranking='player_rating'
)

Exploring the forecasts manually

In [8]:
football_network.get_seasons()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [9]:
app_f = DFGViz.ForecastExplorer(
    network=football_network,
    ratings=["player_rating"]
)

In [10]:
app_f.run('inline')

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
~\Anaconda3\envs\dfg-rating\lib\site-packages\dash\dash.py in dispatch(
    self=<jupyter_dash.jupyter_app.JupyterDash object>
)
   1071         try:
-> 1072             func = self.callback_map[output]["callback"]
        func = undefined
        self.callback_map = {}
        output = '..ratings_chart.figure...teams-ratings.figure..'
   1073         except KeyError:

KeyError: '..ratings_chart.figure...teams-ratings.figure..'

During handling of the above exception, another exception occurred:

KeyError                                  Traceback (most recent call last)
~\Anaconda3\envs\dfg-rating\lib\site-packages\flask\app.py in full_dispatch_request(self=<Flask 'network_explorer'>)
   1948             rv = self.preprocess_request()
   1949             if rv is None:
-> 1950                 rv = self.dispatch_request()
        rv = N

### Inside loop functions

In [47]:
def add_elo_rating(k, n, name):
    n.add_rating(
    rating=ELORating(
        trained=True, 
        rating_name=name,
        **{
            'param_c' : 10,
            'param_d' : 400,
            'param_k' : k,
            'param_w' : 50
        }
    ),
    rating_name=name
)

In [48]:
def aggregate_measures(list_of_matches, prefix='', addBettingMetrics=True):
    if addBettingMetrics:
        return {
            prefix+'_rps': np.mean([m[3]['metrics']['rps'] for m in list_of_matches]),
            prefix+'_likelihood': sum([m[3]['metrics']['likelihood'] for m in list_of_matches]),
            prefix+'_prob_diff': np.mean([m[3]['metrics']['prob_diff'] for m in list_of_matches]),
            prefix+'_draw_prob': np.mean([m[3]['metrics']['draw_prob'] for m in list_of_matches]),
            prefix+'_fav_prob': np.mean([m[3]['metrics']['fav_prob'] for m in list_of_matches]),
            prefix+'_betting_returns': sum([sum([b[0] for b in m[3]['metrics']['betting_returns']]) for m in list_of_matches]),
            prefix+'_betting_returns_expected': sum([sum(b[1] for b in m[3]['metrics']['betting_returns']) for m in list_of_matches]),
            prefix+'_betting_activity': sum([m[3]['metrics']['betting_activity']['qty'] for m in list_of_matches])
        }
    else:
        return {
            prefix+'_rps': np.mean([m[3]['metrics']['rps'] for m in list_of_matches]),
            prefix+'_likelihood': sum([m[3]['metrics']['likelihood'] for m in list_of_matches]),
            prefix+'_prob_diff': np.mean([m[3]['metrics']['prob_diff'] for m in list_of_matches]),
            prefix+'_draw_prob': np.mean([m[3]['metrics']['draw_prob'] for m in list_of_matches]),
            prefix+'_fav_prob': np.mean([m[3]['metrics']['fav_prob'] for m in list_of_matches])
        }

## Betting config

We create a bookmaker and a betting strategy that are going to interact with the generated networks.

In [49]:
bookmaker: BaseBookmaker = factory.new_bookmaker(
    'simple',
    error=factory.new_forecast_error(error_type='factor', error=0.0, scope='positive'),
    margin=factory.new_bookmaker_margin('simple', margin=0.10)
)

In [50]:
betting = FixedBetting(100)

We also create an error rating that we are going to use for the calculated forecasts

In [51]:
rating_error = RatingFunctionError(error='normal', loc=config["rating_error_loc"], scale=config["rating_error_scale"])
rating_error_bookmaker = RatingFunctionError(error='normal', loc=config["bookmaker_rating_error_loc"], scale=config["bookmaker_rating_error_scale"])

NameError: name 'RatingFunctionError' is not defined

### Small loops to test

Small options

In [52]:
k_options = range(15,16,1)
c0_options = np.arange(-1.20, -1.00, 0.1)
c1_options = np.arange(0.0, 0.2, 0.1)
beta_options = np.arange(0.006, 0.008, 0.002)

Full options

In [17]:
k_options = range(10,70,10)
c0_options = np.arange(-1.10, -0.70, 0.1)
c1_options = np.arange(0.1, 0.50, 0.1)
beta_options = np.arange(0.004, 0.012, 0.002)

In [ ]:
experiment_start_time = time.time()
result_list = []
for k in k_options:
    rating_name = f"elo_rating_{k}"
    print(f"Rating <{rating_name}>")
    add_elo_rating(k, football_network, rating_name)
    for c0, c1, beta in itertools.product(c0_options, c1_options, beta_options):
        print(f'Variables: c0: {c0}, c1: {c1} and beta: {beta}')
        forecast_pointer = f"player_{k}_forecast_{c0:.2f}_{c1:.2f}_{beta:.3f}"
        cell_start_time = time.time()
        football_network.add_forecast(
            forecast=LogFunctionForecast(
                outcomes=['home', 'draw', 'away'], 
                coefficients=[c0, c1], 
                beta_parameter=beta
            ),
            forecast_name=forecast_pointer,
            base_ranking=rating_name
        )
        football_network.add_bets(
            bettor_name='b',
            bookmaker='maximumodds',
            betting=betting,
            base_forecast=forecast_pointer
        )
        rps = RankProbabilityScore(outcomes=['home', 'draw', 'away'], forecast_name=forecast_pointer)
        betting_returns = BettingReturnsEvaluator(
            outcomes=['home', 'draw', 'away'], 
            player_name='b', 
            true_model=forecast_pointer, 
            bookmaker_name='maximumodds'
        )
        betting_activity = BettingActivity(outcomes=['home', 'draw', 'away'], player_name='b')
        likelihood = Likelihood(outcomes=['home', 'draw', 'away'], forecast_name=forecast_pointer)
        difference = ProbabilityDifference(outcomes=['home', 'draw', 'away'], forecast_name=forecast_pointer)
        draw_probability = ProbabilityPointer(outcomes=['home', 'draw', 'away'], forecast_name=forecast_pointer, probability_index=1)
        fav_probability = FavouriteProbability(outcomes=['home', 'draw', 'away'], forecast_name=forecast_pointer)

        football_network.add_evaluation([
            (rps, 'rps'),
            (betting_returns, 'betting_returns'),
            (betting_activity, 'betting_activity'),
            (likelihood, 'likelihood'),
            (difference, 'prob_diff'),
            (draw_probability, 'draw_prob'),
            (fav_probability, 'fav_prob')
        ])
        all_matches = [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in football_network.iterate_over_games()]
        result_all = aggregate_measures(all_matches, 'all')
        result_is = aggregate_measures([m for m in all_matches if m[3]['season'] <= 3], 'is')
        result_oos = aggregate_measures([m for m in all_matches if m[3]['season'] > 3], 'oos')
        result = {
            'k': k, 'c0' : c0, 'c1' : c1, 'beta' : beta, **result_is, **result_oos, **result_all
        }
        result_list.append(result)
        print(f"Finished in {float(time.time() - cell_start_time)} seconds")

print(f"Experiment finished in {float(time.time() - experiment_start_time)} seconds with {len(result_list)} observations.")
df = pd.DataFrame(result_list)

Rating <elo_rating_15>


In [ ]:
df

In [20]:
df.to_excel(f"Results_RealWorld.xlsx")